In [1]:
from Gemini.GeminiPredict import GeminiPredict
from Generation.Equation_BaseDataset import Equation, BaseDataset
# from Postprocessing.
from Scraping.Scraper import Scraper

import os
import google.generativeai as genai

### Gemini

In [2]:
api_key = os.getenv("GEMINI_API_KEY")
gp = GeminiPredict(api_key, "Data/HTC_equations.txt", "tunedModels/mmltopython4-f3fuppiemnq9")
gp.predict_from_txt()
gp.save_predictions("Data/HTC_gemini_predictions.txt")

API key loaded successfully!


Generating Predictions: 100%|██████████| 13/13 [00:19<00:00,  1.46s/row]


In [3]:
gp.save_predictions()
# for model in genai.list_tuned_models():
#     print(model.name)

### Generation

In [ ]:
eq = Equation()


In [4]:
class GeminiDataset(BaseDataset):
    def __init__(self, num, filepath):
        super().__init__(num, filepath)

    def get_columns(self):
        return ["text_input", "output"]
    
gd = GeminiDataset(50, "Data/gemini_test_4.csv")
gd.create()

Generating dataset: 100%|██████████| 50/50 [00:00<00:00, 66.17it/s]


### Postprocessing

### Scraping

In [ ]:
sc = Scraper()

### Pseudo Main

- choose DOI to scrape from
- use Scraper class to get MathML equations from paper and save to a txt file
- (also scrape tables for constants and save)
- LLMPredict class takes in MathML and constant files
- predicts python

